# RAG sem langchain
- mais simples e robusto, não tem tantas dependências para ficar dando erro de versão
- processa .pdf e .md
- coloque os arquivos em DOC_FOLDER (ver abaixo) e execute

In [26]:
import os
import glob
import hashlib
import pdfplumber
import numpy as np
import chromadb
from dotenv import load_dotenv
from openai import OpenAI
import time

# ------------------------------
# 1️⃣ Configuration
# ------------------------------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DOC_FOLDER = "C:\\Users\\desig.cateb\\Dados\\R\\testeRAG\\"  # folder containing PDFs and Markdown files
CHUNK_SIZE = 500
CHROMA_COLLECTION_NAME = "rag_docs"
HASH_FILE = "file_hashes.npy"  # store hashes of indexed files

# ------------------------------
# 2️⃣ Initialize OpenAI client
# ------------------------------
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# ------------------------------
# 3️⃣ Embedding function (Chroma v0.4+ compatible)
# ------------------------------
class OpenAIEmbeddingFunction:
    def __call__(self, input: list[str]) -> list[list[float]]:
        """Used for adding documents / embedding"""
        return self._embed(input)

    def embed_query(self, input: list[str]) -> list[list[float]]:
        """Used for querying"""
        return self._embed(input)

    def _embed(self, texts: list[str]) -> list[list[float]]:
        embeddings = []
        for text in texts:
            resp = client.embeddings.create(
                model="text-embedding-3-large",
                input=text
            )
            embeddings.append(np.array(resp.data[0].embedding, dtype=np.float32))
        return embeddings

embedding_function = OpenAIEmbeddingFunction()



# ------------------------------
# 4️⃣ Initialize Chroma DB
# ------------------------------
chroma_client = chromadb.Client()

if CHROMA_COLLECTION_NAME in [c.name for c in chroma_client.list_collections()]:
    collection = chroma_client.get_collection(CHROMA_COLLECTION_NAME)
else:
    collection = chroma_client.create_collection(
        name=CHROMA_COLLECTION_NAME,
        embedding_function=embedding_function
    )


In [27]:
# ------------------------------
# 5️⃣ Load previous file hashes
# ------------------------------
if os.path.exists(HASH_FILE):
    file_hashes = np.load(HASH_FILE, allow_pickle=True).item()
else:
    file_hashes = {}

In [28]:
# ------------------------------
# 6️⃣ Helper functions
# ------------------------------
def split_text(text, chunk_size=CHUNK_SIZE):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def get_file_hash(path):
    """Compute a hash of a file to detect changes"""
    hasher = hashlib.md5()
    with open(path, "rb") as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

def extract_text(path):
    """Extract text from PDF or Markdown"""
    if path.endswith(".pdf"):
        with pdfplumber.open(path) as pdf:
            return "\n\n".join(page.extract_text() or "" for page in pdf.pages)
    elif path.endswith(".md"):
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    else:
        return ""


In [55]:
# ------------------------------
# 7️⃣ Find new or updated files
# ------------------------------
all_files = glob.glob(os.path.join(DOC_FOLDER, "*.pdf")) + glob.glob(os.path.join(DOC_FOLDER, "*.md"))
new_or_updated_files = []

for fpath in all_files:
    file_hash = get_file_hash(fpath)
    if fpath not in file_hashes or file_hashes[fpath] != file_hash:
        new_or_updated_files.append((fpath, file_hash))

print(f"Found {len(new_or_updated_files)} new or updated files.")


Found 1 new or updated files.


In [56]:
# ------------------------------
# 8️⃣ Embed and add to Chroma
# ------------------------------
total_chunks = 0

for fpath, file_hash in new_or_updated_files:
    text = extract_text(fpath)
    chunks = split_text(text, CHUNK_SIZE)
    
    ids = [f"{os.path.basename(fpath)}_chunk_{i}" for i in range(len(chunks))]
    metadatas = [{"source": os.path.basename(fpath), "chunk": i} for i in range(len(chunks))]
    
    collection.add(documents=chunks, metadatas=metadatas, ids=ids)
    file_hashes[fpath] = file_hash
    total_chunks += len(chunks)

print(f"Embedded {total_chunks} chunks from new/updated files.")

# Save updated hashes
np.save(HASH_FILE, file_hashes)

Embedded 2 chunks from new/updated files.


In [57]:
# ------------------------------
# 9️⃣ Query / Retrieval
# ------------------------------
query = "Explique o que foi o pl antifaccção votado na câmara"

results = collection.query(
    query_texts=[query],
    n_results=3
)

context_text = "\n\n".join(results['documents'][0])
print (context_text)

23/11/2025, 11:19 A mira errada do PL Antifacção: mais votos, menos segurança - Estadão Estadão / Política / Blogs Blog do Fausto Macedo Notícias e artigos do mundo do Direito: a rotina da Polícia, Ministério Público e Tribunais Seguir Opinião A mira errada do PL Antifacção: mais votos, menos segurança No fim das contas, trocamos inteligência por truculência, estratégia por marketing político. Corremos o risco de superlotar ainda mais os presídios, transformando-os em centros de recrutamento ainda mais eficientes para o crime, sem de fato enfraquecer o poder paralelo que domina comunidades inteiras PUBLICIDADE Convidado file:///C:/Users/User/Downloads/a-mira-errada-do-pl-antifaccao-mais-votos-menos-seguranca.html 1/7 23/11/2025, 11:19 A mira errada do PL Antifacção: mais votos, menos segurança - Estadão Por Jacqueline Valle 19/11/2025 | 17h50 Aprovado na Câmara dos Deputados em meio a aplausos e discursos de firmeza, o chamado “PL Antifacção” chega com a promessa de ser a resposta defi

In [66]:
# ------------------------------
# 🔟 Generate answer
# ------------------------------
system_prompt = """
You are a helpful assistant. Write in portuguese with the simplest language possible. Answer the user’s questions using the provided context. DO NOT DEDUCE ANY EXPLANATION THAT IS NOT IN THE CONTEXT. 
If the context do not have information to answer the question, ignore the rest of this prompt.
The answer should provide this information:
1) votacao: inform when the proposta/lei was approved, formatted dd/mm/YYYY. if the article does not mention, inform the article date. 
    if possible, inform a code or number of the proposicao or PL (projeto de lei) or PEC (proposta de emenda à constituição)
    or any code that helps identify it. If there is no code, do not mention anything about it. Mention the number of votes for and against it. 
2) titulo: provide a title explaining what this is about in up to 100 characters;
3) resumo: do not repeat information provided in items (1) or (2). Provide an abstract of what was approved in no more than 500 characters.  Use a concise and simple language. 
4) explicacao: create a thorough explanation about the question in aproximately 2000 characters. Whenever more technical expressions are used (e.g., trânsito em julgado, ), avoid it or briefly explain them in parentheses right afterwards. Try to elaborate on who gains and who loses with it, how it can affect employment, its economic and political implications, etc. Explain how it is important or harmful to the country. At the end, briefly mention the sources of the information, including author, where and date.
"""

user_prompt = f"""
Context:
{context_text}

Question:
{query}
"""

completion = client.chat.completions.create(
    model="gpt-4o-mini",

    # Messages
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],

    # 🔹 Forces deterministic output (0 = no randomness)
    temperature=0.0,

    # 🔹 Alternative randomness control (lower = more deterministic)
    top_p=1.0,              # 1.0 = disabled, use full probability distribution

    # 🔹 Makes the model avoid extremely long wandering responses
    # max_tokens=500,         # Adjust based on expected output length

    # 🔹 Ensures output format follows instructions strictly
    #     Setting it low reduces the amount of "creative" deviation.
    presence_penalty=0.0,   # Do not encourage new topics
    #frequency_penalty=0.0,  # Do not penalize repeating necessary words

    # 🔹 Helps avoid timeouts and partial responses on slow networks
    timeout=30.0,           # Seconds – decide based on your environment

    # 🔹 Makes the model restart generation if output is incomplete
    #     Useful in production systems that require complete responses.
    n=1,                    # Number of completions to generate (1 = simplest)

    # 🔹 Helps debugging and monitoring consistency
    #     Using a seed forces reproducible output across identical requests.
    seed=1234,              # Any integer → deterministic sampling (when temperature > 0)

    # 🔹 Tool and function calling (disabled here, but included for clarity)
    tools=[],               # Add function definitions here (optional)
    tool_choice="none",     # Prevent tool-calling randomness
)


print("\n--- Generated Answer ---\n")
print(completion.choices[0].message.content)



--- Generated Answer ---

1) **votacao**: A proposta foi aprovada na Câmara dos Deputados em 19/11/2025. O número de votos a favor foi 51 e contra 17.

2) **titulo**: PL Antifacção: Medidas severas contra o crime organizado.

3) **resumo**: O PL Antifacção foi aprovado com a promessa de combater o crime organizado, aumentando penas e endurecendo regras. No entanto, críticos afirmam que ele é mais uma medida populista que não ataca as raízes do problema, como o financiamento das facções. O projeto pode levar a abusos legais e não garante segurança real.

4) **explicacao**: O PL Antifacção é uma proposta que visa endurecer as penas para crimes relacionados a facções criminosas, com a intenção de fortalecer a segurança pública. No entanto, muitos especialistas e críticos argumentam que essa abordagem é superficial e não resolve os problemas estruturais que alimentam o crime organizado. A proposta inclui penas que podem chegar a 66 anos e mudanças na progressão de regime, mas ignora a que